<a href="https://colab.research.google.com/github/Michwynn/Capstone-Team-Air-/blob/Data-Cleaning/Cleaning_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import warnings # supress warnings
warnings.filterwarnings('ignore')

# data manipulation
import pandas as pd
import numpy as np 
import re # regex
pattern = '\w+' # default regex pattern
import random
import string
from collections import Counter, defaultdict
from itertools import chain
from functools import reduce

# Language Detection
!pip install fasttext
import fasttext

# helper function to remove HTML tags from comments
from bs4 import BeautifulSoup

# mount google drive dataset
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Capstone
pd.set_option('display.max_columns', None)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Capstone


**Reviews dataset**

Preliminary cleaning and language detection

In [ ]:
%%time
# ### read datafile
# reviews = pd.read_csv('assets/raw_assets/reviews.csv')
# ### Drop the null value and comments value with "."
# reviews = reviews[reviews['comments'].notna()]
# ### Drop the rows with empty string for comments column
# reviews = reviews.loc[reviews["comments"] != "."]
# reviews['comments'] = reviews['comments'].apply(lambda s: BeautifulSoup(s).text)

# # removing special characters including punctuations and emojis. 
# reviews['comments'] = reviews['comments'].str.replace('[^\w\s#@/:%.,_-]', '', flags = re.UNICODE)
# reviews.columns = reviews.columns.str.strip() 

# # setting up the pretrained model
# fasttxt_model = fasttext.load_model("assets/raw_assets/lid.176.bin")
# reviews['lang'] = reviews['comments'].apply(lambda x: fasttxt_model.predict(x)[0][0][-2:])

# # calculating number of rows with non-english comments 
# display(reviews.lang.value_counts())
# prop = (1214643-1089747)/1214643*100
# f'The total non-english comments proportion is {prop}.'

# # keeping only rows with English comments
# reviews = reviews[reviews['lang'] == 'en']

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 11 µs


**VADER Sentiment Tagging**

The result generated by VADER is a dictionary of 4 keys neg, neu, pos and compound:

neg, neu, and pos meaning negative, neutral, and positive respectively. Their sum should be equal to 1 or close to it with float operation.

compound corresponds to the sum of the valence score of each word in the lexicon and determines the degree of the sentiment rather than the actual value as opposed to the previous ones. Its value is between -1 (most extreme negative sentiment) and +1 (most extreme positive sentiment). Using the compound score can be enough to determine the underlying sentiment of a text, because for:

a positive sentiment, compound ≥ 0.05

*   a positive sentiment, compound ≥ 0.05
*   a negative sentiment, compound ≤ -0.05
*   a neutral sentiment, the compound is between [-0.05, 0.05]

In [ ]:
%%time
# reviews['VADER_compound_score'] = [round(SentimentIntensityAnalyzer().polarity_scores(x)['compound'], 2) for x in reviews['comments']]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.63 µs


**Listings dataset**

In [ ]:
### read datafile
Listings = pd.read_csv('assets/raw_assets/listings.csv')
### select required columns
Listings = Listings[['id', 'neighbourhood_cleansed', 'latitude', 'longitude', 'host_since', 'host_is_superhost',
                     'host_has_profile_pic', 'host_identity_verified', 'amenities', 
                     'property_type', 'room_type', 'accommodates','minimum_nights', 'maximum_nights',
                     'number_of_reviews', 'price']]
# create new column that counts total no of amenities
Listings['amenities_count'] = Listings['amenities'].str.count(',')

**Crime dataset**

In [ ]:
### read datafile
crime_df = pd.read_csv('assets/raw_assets/crime.csv')
### summing all crimes
crime_df = crime_df.groupby('LookUp_BoroughName').agg('sum').reset_index()
crime_df['total_crimes'] = crime_df.sum(axis=1)
### renaming to borough 
crime_df = crime_df[['LookUp_BoroughName','total_crimes']].rename(columns={'LookUp_BoroughName':'Borough'})

### Assumption taken here -> London Heathrow and London City Airports = City of London
crime_df['Borough'] = crime_df['Borough'].replace(['London Heathrow and London City Airports'], 'City of London') 
crime_df.head(5)

,Borough,total_crimes
0,Barking and Dagenham,38988
1,Barnet,55331
2,Bexley,30704
3,Brent,57075
4,Bromley,44662


**Population dataset**

In [ ]:
### read datafile
population_df = pd.read_csv('assets/raw_assets/population.csv')
### rename columns
population_df = population_df.rename(columns = {'Area name': 'Borough'})
### select required columns
population_df = population_df[['Borough', 'Population per square kilometre', 'Census population']]
population_df.head(5)

,Borough,Population per square kilometre,Census population
0,City of London,2854.403716,7400
1,Barking and Dagenham,6214.914626,185900
2,Barnet,4784.427280,356400
3,Bexley,4268.804035,232000
4,Brent,8067.585607,311200


**Property prices dataset**

Source: https://data.london.gov.uk/dataset/average-house-prices

In [ ]:
### read datafile
property_df = pd.read_csv('assets/raw_assets/propertyprices.csv', encoding = "ISO-8859-1")
property_df = property_df.iloc[: , :2]
### rename columns
property_df.columns = ['Borough','Property price']
### price column to correct dtype
property_df['Property price'] = property_df['Property price'].replace('\D', '', regex = True).astype(int)
### City of Westminster = Westminster
property_df['Borough'] = property_df['Borough'].replace(['City of Westminster'],'Westminster')
### select required columns
property_df.head(5)

,Borough,Property price
0,Barking and Dagenham,335683
1,Barnet,577319
2,Bexley,383366
3,Brent,494976
4,Bromley,479845


**Places of Interest dataset**

In [ ]:
interests_df = pd.read_csv('assets/raw_assets/visitor.csv', index_col = 0, encoding = "ISO-8859-1")
### Rename columns
interests_df = interests_df.set_axis([ 'Site', 'Total Visits', 'Charge/ free', 'In/ Outdoor', '% +/-', 'Area', 'Group', 'Note', 'Latitude', 'Longitude', 'Borough'], axis=1, inplace = False)
### Filter Points of Interest for only London
interests_df = interests_df[interests_df['Area']=='London']

Haversine - distance calculation

In [ ]:
lat_long_data = np.array(list(zip(Listings['latitude'], Listings['longitude'])))
poi_lat_long = np.array(list(zip(interests_df['Latitude'], interests_df['Longitude'])))

#### Convert latitude longitude from degrees to Radians
data1 = np.deg2rad(lat_long_data)
data2 = np.deg2rad(poi_lat_long)

#### Keep latitudes and longitudes to separate array 
lat1 = data1[:,0]
lng1 = data1[:,1]
lat2 = data2[:,0]
lng2 = data2[:,1]

diff_lat = lat1[:,None] - lat2
diff_lng = lng1[:,None] - lng2
d = np.sin(diff_lat/2)**2 + np.cos(lat1[:,None])*np.cos(lat2) * np.sin(diff_lng/2)**2
dist_final = 2 * 6371 * np.arcsin(np.sqrt(d))
min_dist = list(np.min(dist_final, axis = 1))
dist_5k_cnt = list(np.sum(dist_final < 5, axis = 1))
final_data = list(zip(min_dist, dist_5k_cnt))
interests_df = pd.DataFrame(final_data, columns = ['closest_POI_dist', 'within_5k_POI'])
interests_df.head(5)

,closest_POI_dist,within_5k_POI
0,4.087250,3
1,1.018278,20
2,1.249044,6
3,1.297479,10
4,3.378689,1


**London Underground data**

In [ ]:
### London underground data 
tube_data = pd.read_html('https://commons.wikimedia.org/wiki/London_Underground_geographic_maps/Tables')
tube_data = tube_data[0]

Haversine - distance calculation

In [ ]:
### Convert lat-long to an array
lat_long_data = np.array(list(zip(Listings['latitude'], Listings['longitude'])))
tube_lat_long = np.array(list(zip(tube_data['latitude'], tube_data['longitude'])))

### Convert latitude longitude from degrees to Radians
data1 = np.deg2rad(lat_long_data)
data2 = np.deg2rad(tube_lat_long)

### Keep latitudes and longitudes to separate array 
lat1 = data1[:,0]
lng1 = data1[:,1]

lat2 = data2[:,0]
lng2 = data2[:,1]

### Compute a 67903 X 307 array with each row containing the distance in Km of each Airbnb property from each of the 307 tube
### Stations
diff_lat = lat1[:,None] - lat2
diff_lng = lng1[:,None] - lng2
d = np.sin(diff_lat/2)**2 + np.cos(lat1[:,None])*np.cos(lat2) * np.sin(diff_lng/2)**2
dist_final = 2 * 6371 * np.arcsin(np.sqrt(d))

### find the closest station for each row
min_dist = list(np.min(dist_final, axis = 1))

### find number of stations within 1 km
dist_1k_cnt = list(np.sum(dist_final < 1, axis = 1))
final_data = list(zip(min_dist, dist_1k_cnt))
data_stations = pd.DataFrame(final_data, columns = ['closest_station_dist', 'within_1k_station'])
data_stations

,closest_station_dist,within_1k_station
0,0.651237,1
1,0.803020,2
2,0.035661,3
3,1.457888,0
4,0.172913,1
...,...,...
71933,11.642491,0
71934,0.299767,2
71935,0.942688,1
71936,0.818499,3


**Distance from city center of London**

In [ ]:
ldn_center_lat_long = np.array([51.5072, -0.1276])
data = np.deg2rad(ldn_center_lat_long)

lat = data[0]
lng = data[1]

diff_lat = lat1[:,None] - lat
diff_lng = lng1[:,None] - lng

d = np.sin(diff_lat/2)**2 + np.cos(lat1[:,None])*np.cos(lat) * np.sin(diff_lng/2)**2
dist_final = 2 * 6371 * np.arcsin(np.sqrt(d))

data_center = pd.DataFrame(dist_final, columns = ['dist_from_center'])
data_center

,dist_from_center
0,6.905804
1,3.539090
2,5.720974
3,6.731165
4,9.112363
...,...
71933,20.895693
71934,16.232251
71935,10.214938
71936,2.470862


**Merge dataframes**

In [ ]:
# joining no of stations/distance + no of attractions/distance with listings df + distance from city center
df_final = pd.concat([Listings, data_stations], axis = 1)
df_final = pd.concat([df_final, interests_df], axis = 1)
df_final = pd.concat([df_final, data_center], axis = 1)

# compile the list of dataframes that have borough and merge each of them
data_frames = [crime_df, population_df, property_df]
df_secondary = reduce(lambda  left,right: pd.merge(left, right,on = ['Borough'], how = 'outer'), data_frames)
# join with listings df
df = pd.merge(df_secondary, df_final,  how ='inner', left_on = ['Borough'], right_on = ['neighbourhood_cleansed'])
# drop na
df = df.dropna()
# clean target variable - price column
df['price'] = df['price'].str.replace("$", "")
df['price'] = df['price'].str.replace(",", "")

**# Write out csv**

In [ ]:
df.to_csv('assets/cleaned_assets/df_cleaned.csv')